In [1]:
data <- read.csv("../input/mooc-dataset/big_student_clear_third_version.csv", header=TRUE)

In [42]:
install.packages("arules")
install.packages("arulesViz")
install.packages("wordcloud")
library(arules)
library(arulesViz)
library(wordcloud)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: grid

Registered S3 method overwritten by 'seriation':
  method         from 
  reorder.hclust gclus

Loading required package: RColorBrewer



In [2]:
install.packages("rJava")
install.packages("TunePareto")
install.packages("rCBA")
library("rJava")
library("TunePareto")
library("rCBA")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: arules

Loading required package: Matrix


Attaching package: ‘arules’


The following objects are masked from ‘package:base’:

    abbreviate, write




In [4]:
# 1단계
Institute <- data$institute
Course <- data$course_id
Region <- data$final_cc_cname_DI
Degree <- data$LoE_DI
# 2단계
#공백처리
Region <- factor(gsub(' ','',Region)) 
#OtherNorth&CentralAmer.,Caribbean의 ','가 csv변환시 문제가 되어 제거
Region <- factor(gsub(',','',Region)) 
# less than Secondary가 less로 잘리는 문제 해결
Degree <- factor(gsub(' ','',Degree))
RawTransactions <- paste(Institute,Course,Region,Degree,sep="_")

In [18]:
train <- data.frame(data[,c("institute","course_id","final_cc_cname_DI","LoE_DI")], check.names=FALSE)
train[,"Rawtransacations"] <- RawTransactions
train[,"Rawtransacations"] <- as.factor(train[,"Rawtransacations"])
txns <- as(train[1:20000,],"transactions")

In [85]:
options(java.parameters = "-Xmx8048m")
memory.limit(size=10000000024)

Warning message:
“'memory.limit()' is Windows-specific”


[1] Inf

In [86]:
gc()
J("java.lang.Runtime")$getRuntime()$gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2786923,148.9,7432494,397.0,11613270,620.3
Vcells,11313396,86.4,36377684,277.6,660255662,5037.4


In [ ]:
train <- data.frame(data[,c("institute","course_id","final_cc_cname_DI","LoE_DI")], check.names=FALSE)
train[,"Rawtransacations"] <- RawTransactions
train[,"Rawtransacations"] <- as.factor(train[,"Rawtransacations"])
txns <- as(train[1:20000,],"transactions")

In [70]:
fp_rules = rCBA::fpgrowth(txns, support=0.005, confidence=0.05, maxLength=3, 
                       consequent="Rawtransacations",parallel=TRUE)

2020-06-11 07:01:10 rCBA: initialized

2020-06-11 07:01:20 rCBA: data 20000x1459

	 took: 10.15  s

2020-06-11 07:01:20 rCBA: rules 223

	 took: 0.13  s



In [75]:
inspect(sort(fp_rules, by="support")[2])

    lhs                     rhs                                                        support confidence lift
[1] {institute=HarvardX} => {Rawtransacations=HarvardX_PH207x_UnitedStates_Bachelor's}  0.0641     0.0641    1


In [76]:
inspect(sort(fp_rules, by="confidence")[1])

    lhs                          rhs                                                support confidence     lift
[1] {final_cc_cname_DI=India,                                                                                  
     LoE_DI=Doctorate}        => {Rawtransacations=HarvardX_PH207x_India_Doctorate}  0.0077  0.9447853 122.6994


In [77]:
inspect(sort(fp_rules, by="lift")[1])

    lhs                          rhs                                                support confidence     lift
[1] {final_cc_cname_DI=India,                                                                                  
     LoE_DI=Doctorate}        => {Rawtransacations=HarvardX_PH207x_India_Doctorate}  0.0077  0.9447853 122.6994


In [78]:
train <- data.frame(data[,c("institute","course_id","final_cc_cname_DI","LoE_DI")], check.names=FALSE)
txns <- as(train[1:20000,],"transactions")

In [79]:
gc()
J("java.lang.Runtime")$getRuntime()$gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2801031,149.6,7432494,397,11613270,620.3
Vcells,11337904,86.6,45472104,347,660255662,5037.4


In [80]:
fp_rules = rCBA::fpgrowth(txns, support=0.005, confidence=0.05, maxLength=3, 
                       consequent="final_cc_cname_DI",parallel=TRUE)

2020-06-11 07:02:10 rCBA: initialized

2020-06-11 07:02:19 rCBA: data 20000x54

	 took: 9.11  s

2020-06-11 07:02:19 rCBA: rules 93

	 took: 0.11  s



In [82]:
inspect(sort(fp_rules, by="support")[2])

    lhs                     rhs                               support confidence lift
[1] {institute=HarvardX} => {final_cc_cname_DI=United States} 0.36665    0.36665    1


In [83]:
inspect(sort(fp_rules, by="confidence")[1])

    lhs                             rhs                               support confidence     lift
[1] {LoE_DI=Less than Secondary} => {final_cc_cname_DI=United States} 0.01415  0.6690307 1.824712


In [84]:
inspect(sort(fp_rules, by="lift")[1])

    lhs                  rhs                               support confidence
[1] {course_id=CB22x} => {final_cc_cname_DI=Unknown/Other} 0.0071  0.07033185
    lift    
[1] 6.762678
